In [3]:
#pip install pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in e:\ds440proj\btcpred\.venv\lib\site-packages (2.2.0)




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [101]:
	from bs4 import BeautifulSoup #module for web scraping install by pip install beautifulsoup4
	import requests #for requesting html. install by pip install requests
	from requests.adapters import HTTPAdapter
	from requests.packages.urllib3.util.retry import Retry
	import re #regular expression for data extraction by pattern matching. installed by default.
	import pandas as pd # for dataframe. install by pip install pandas
	from csv import reader #for list structure. installed by default.
	from tqdm import tqdm
	import os
	import shutil
	import uuid
	import time
	
	start_date = '2017/08/01'
	end_date = '2024/04/01'
	
	dir = str(uuid.uuid4())
	if os.path.exists(dir):
	    shutil.rmtree(dir)
	os.makedirs(dir)
	features=pd.DataFrame({'features':['transactions','size','sentbyaddress','difficulty','hashrate','mining_profitability','sentinusd','transactionfees','median_transaction_fee','confirmationtime','transactionvalue','mediantransactionvalue','activeaddresses','top100cap','fee_to_reward','price']})
	indicators=pd.DataFrame({'indicators':['sma','ema','wma','trx','mom','std','var','rsi','roc']})
	periods=pd.DataFrame({'periods':['3','7','14','30','90']})
	crypto=pd.DataFrame({'crypto':['btc']})
	df=pd.concat([crypto, features,indicators,periods], axis=1)

	#for raw values
	#all kinds of fees and transaction values are in USD. divide by price USD to obtain BTC
	url_list=[] #stores generated urls
	feature_list=[] #store feature names
	i=0
	while (i<=15): #this loop generates urls for raw values
		url='https://bitinfocharts.com/comparison/'+df['features'][i]+'-'+'btc'+'.html'
		feature = df['features'][i]
		if "fee" in feature:
			feature=df['features'][i]+'USD'
		if 'value' in feature:
			feature=df['features'][i]+'USD'
		if 'usd' in feature:
			feature=df['features'][i]+'USD'
		url_list.append(url)
		feature_list.append(feature)
		#print(feature,' ',url)
		i=i+1
	#for indicators
	#all kinds of fees and transaction values are in USD. drop them or recalculate them after converting the raw values to the BTC
	i=0
	while (i<=15): #this nested while loop generates url structure for all the indicators. for other currencies change btc to CURRENCY_NAME
		j=0
		while (j<=8):
			k=0
			while (k<=4):
				url='https://bitinfocharts.com/comparison/'+df['features'][i]+'-'+'btc'+'-'+df['indicators'][j]+df['periods'][k]+'.html'
				feature=df['features'][i]+df['periods'][k]+df['indicators'][j]
				if "fee" in feature:
					feature=df['features'][i]+df['periods'][k]+df['indicators'][j]+'USD'
				if 'value' in feature:
					feature=df['features'][i]+df['periods'][k]+df['indicators'][j]+'USD'
				if 'price' in feature:
					feature=df['features'][i]+df['periods'][k]+df['indicators'][j]+'USD'
				if 'usd' in feature:
					feature=df['features'][i]+df['periods'][k]+df['indicators'][j]+'USD'
				if 'fee_in_reward' in feature:
					feature=df['features'][i]+df['periods'][k]+df['indicators'][j]
				url_list.append(url)
				feature_list.append(feature)
				#print(feature,' ',url)
				k=k+1
			j=j+1
		i=i+1
	




In [102]:
df


,crypto,features,indicators,periods
0,btc,transactions,sma,3
1,NaN,size,ema,7
2,NaN,sentbyaddress,wma,14
3,NaN,difficulty,trx,30
4,NaN,hashrate,mom,90
5,NaN,mining_profitability,std,NaN
6,NaN,sentinusd,var,NaN
7,NaN,transactionfees,rsi,NaN
8,NaN,median_transaction_fee,roc,NaN
9,NaN,confirmationtime,NaN,NaN


In [103]:
	df_feature=pd.DataFrame(feature_list,columns=['Features']) # convert feature list to dataframe
	df_url=pd.DataFrame(url_list,columns=['URL']) # convert url list to dataframe

	df2=df_feature.join(df_url) # join the feature and url dataframes 
	
	features=pd.DataFrame(columns=df2.Features) #change the feature list to columns



In [104]:
	columns=len(features.columns) #to be used in while loop for getting data
	columns

	


736

In [105]:
i

16

In [106]:
features

Features,transactions,size,sentbyaddress,difficulty,hashrate,mining_profitability,sentinusdUSD,transactionfeesUSD,median_transaction_feeUSD,confirmationtime,...,price3rsiUSD,price7rsiUSD,price14rsiUSD,price30rsiUSD,price90rsiUSD,price3rocUSD,price7rocUSD,price14rocUSD,price30rocUSD,price90rocUSD


In [107]:
date = []  # List to store the date columns
print('Building Date Columns ...')
for column_name in tqdm(features.columns):
    date_column_name = column_name + 'Date'  # Create the date column name
    features[date_column_name] = date_column_name  # Assign the date column to the DataFrame
    date.append(date_column_name) 

Building Date Columns ...


  0%|          | 0/736 [00:00<?, ?it/s]C:\Users\13275\AppData\Local\Temp\ipykernel_36108\2238912588.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[date_column_name] = date_column_name  # Assign the date column to the DataFrame
C:\Users\13275\AppData\Local\Temp\ipykernel_36108\2238912588.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[date_column_name] = date_column_name  # Assign the date column to the DataFrame
C:\Users\13275\AppData\Local\Temp\ipykernel_36108\2238912588.py:5: PerformanceWarning: Data

In [92]:
features

Features,transactions,size,sentbyaddress,difficulty,hashrate,mining_profitability,sentinusdUSD,transactionfeesUSD,median_transaction_feeUSD,confirmationtime,...,price3rsiUSDDate,price7rsiUSDDate,price14rsiUSDDate,price30rsiUSDDate,price90rsiUSDDate,price3rocUSDDate,price7rocUSDDate,price14rocUSDDate,price30rocUSDDate,price90rocUSDDate


In [99]:
i=0
columnNames=[features.columns[i+columns],features.columns[i]]
url = df2.URL[i]
session = requests.Session()
retry = Retry(connect=10, backoff_factor=3)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
page=session.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
values=soup.find_all('script')[6].get_text()
newval=values.replace('[new Date("','')
newval2=newval.replace('"),',";")
newval3=newval2.replace('],',',')
newval4=newval3.replace('],',']]')
newval5=newval4.replace('null','0')
x = re.findall('\\[(.+?)\\]\\]', newval5)
df3=pd.DataFrame(list(reader(x)))
df_transposed=df3.transpose()
df_transposed.columns=['Value']
df_new = df_transposed['Value'].str.split(';', expand=True)
df_new.columns= columnNames
mask = (df_new[features.columns[i+columns]] >= start_date) & (df_new[features.columns[i+columns]] <= end_date)
df_new= df_new.loc[mask]
features[features.columns[i]] = df_new[features.columns[i]]
features[features.columns[i+columns]]= df_new[features.columns[i+columns]]
df_new.columns = df_new.columns.str.replace('.*Date.*', 'Date', regex=True)
path=dir+'/'+features.columns[i]+'.csv'
df_new.set_index('Date', inplace=True)
df_new.to_csv(path,sep=',', columns=[features.columns[0]])

In [100]:
df_new

,transactions
Date,
2017/08/01,131726
2017/08/02,209169
2017/08/03,212879
2017/08/04,235613
2017/08/05,239594
...,...
2024/03/28,428518
2024/03/29,449277
2024/03/30,489917


In [108]:
i=0
print('Requesting data ... ')
for i in tqdm(range(columns)): #the most important. getting the data from the website. DONT ABUSE IT. you might be IP banned for requesting a lot
	columnNames=[features.columns[i+columns],features.columns[i]]
	url = df2.URL[i]
	session = requests.Session()
	retry = Retry(connect=10, backoff_factor=3)
	adapter = HTTPAdapter(max_retries=retry)
	session.mount('http://', adapter)
	session.mount('https://', adapter)
	page=session.get(url)
	#page = requests.get(url, time.sleep(3),timeout=10)
	#print(page)
	soup = BeautifulSoup(page.content, 'html.parser')
	values=soup.find_all('script')[6].get_text()
	newval=values.replace('[new Date("','')
	newval2=newval.replace('"),',";")
	newval3=newval2.replace('],',',')
	newval4=newval3.replace('],',']]')
	newval5=newval4.replace('null','0')
	x = re.findall('\\[(.+?)\\]\\]', newval5)
	df3=pd.DataFrame(list(reader(x)))
	df_transposed=df3.transpose()
	df_transposed.columns=['Value']
	df_new = df_transposed['Value'].str.split(';', expand=True)
	df_new.columns= columnNames
	mask = (df_new[features.columns[i+columns]] >= start_date) & (df_new[features.columns[i+columns]] <= end_date)
	df_new= df_new.loc[mask]
	features[features.columns[i]] = df_new[features.columns[i]]
	features[features.columns[i+columns]]= df_new[features.columns[i+columns]]
	df_new.columns = df_new.columns.str.replace('.*Date.*', 'Date', regex=True)
	path=dir+'/'+features.columns[i]+'.csv'
	df_new.set_index('Date', inplace=True)
	df_new.to_csv(path,sep=',', columns=[features.columns[i]])
	#print(df_new)
	#i = i+1



Requesting data ... 


100%|██████████| 736/736 [06:58<00:00,  1.76it/s]


In [109]:
i=0
pricepath=dir+'/'+'price.csv'
df_merge=pd.read_csv(pricepath,sep=',')
print('Processing files ... ')
for i in tqdm(range(columns)):
	path=dir+'/'+features.columns[i]+'.csv'
	df=pd.read_csv(path,sep=',')
	df_merge = pd.merge(df_merge, df, left_on='Date', right_on='Date')
	#i = i+1
	
df_merge.drop(columns=['price_y'], inplace=True)
df_merge.columns = df_merge.columns.str.replace('price_x', 'priceUSD')

df_merge
	
path2=dir+'/'+'Merged_Unconverted_BTC_Data.csv'
df_merge.to_csv(path2,sep=',')
df_all = pd.read_csv(path2,sep = ',')
df_all.columns
	
	

df_all.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

df_all['priceUSD']=df_merge['priceUSD']
df_all
	
filename=dir+'_'+'BTC_Data.csv'
df_all.to_csv(filename,sep=',')
	
#queue.put(df_all)


Processing files ... 


100%|██████████| 736/736 [00:04<00:00, 167.81it/s]
